In [1]:
from pydantic import BaseModel
import instructor
from openai import AsyncOpenAI
from rich import print

client = instructor.from_openai(
    client=AsyncOpenAI(),
)


class GeneratedStory(BaseModel):
    setting: str
    plot_summary: str
    art_style: str
    main_character_description: str
    choices: list[str]


story = await client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": """
        
        Generate a story set in a post-apocalyptic world where zombines have taken over the world. The user encounters the last human alive!

        Flesh out a description of the setting and the plot summary that is vivid and has a lot of details
        
        Rules
        - Generate 2-4 choices that represent actions which the user can take and make sure it's not a terminal choice, the story is just beginning!
        - Generate a visual description that can be used to generate an image for the story
        - Art style should just describe the style of the image. This should mention the color palette, the style of the characters, and the style of the background.
        """,
        }
    ],
    model="gpt-4o-mini",
    response_model=GeneratedStory,
)

print(story)

GeneratedStory(
    setting='The world lies in ruins, a once-vibrant city now transformed into a cavern of decay and despair. 
Skyscrapers, formerly symbols of human achievement, stand abandoned, their skeletal frames intertwined with 
overgrown vines and moss. Streets are littered with remnants of the past; shattered glass, rusted vehicles, and the
eerie silence that follows a catastrophe. The sky is a constant swirl of dark gray clouds, heavy with the threat of
rain, casting a perpetual gloom over the remnants of civilization. In this dire landscape, zombines—zombie-like 
creatures with deteriorating animalistic features—roam the streets, drawn by the slightest sound or movement. Their
grotesque forms lurch ominously among the ruins, a chilling reminder of what humanity has lost. In the center of 
this chaos, a lone figure stands; the last surviving human, a testament to resilience in a world gone mad.',
    plot_summary='As the sun sets behind the skeletal remains of skyscrapers, you find yourself navigating the 
treacherous terrain of your former city, a place now overrun by zombines. You have always been cautious, keeping to
the shadows and scavenging for supplies. Today, however, fate takes a curious turn. While rummaging through the 
debris of a dilapidated supermarket, you hear a faint rustling sound behind a toppled shelf. Heart pounding, you 
grab your makeshift weapon—a rusted pipe—and prepare for the worst. To your astonishment, instead of a zombine, you
discover a frail but sturdy woman, the last human survivor, hiding from the hoard. She has a wild look in her eye, 
marked by streaks of dirt and a tattered jacket. As you introduce yourself, she reveals that she’s been living in 
the shadows, avoiding the zombines and scavenging what little remains of food. Together, you decide to find a way 
to survive, but first, a choice must be made: Will you enlist her help, go it alone, or plan to clear the area of 
zombines?',
    art_style='The art style is dark and gritty, with a muted color palette dominated by browns, grays, and hints 
of dark green, reflecting the decay of the environment. Characters are illustrated with exaggerated features—sharp 
angles for the zombines to accentuate their menace, while the last human has softer, yet weary traits that depict 
resilience. The background is richly detailed, showcasing crumbling buildings, overgrowth, and a haunting 
atmosphere that evokes a sense of dread and survival.',
    main_character_description='The main character is a scrappy survivor in their late twenties, with dirt-smudged 
skin and unkempt hair. They wear a patchwork of rugged clothing, a mix of scavenged armor and practical wear, 
reflecting their resourcefulness and adaptability in the treacherous environment. Their eyes are sharp and 
determined, betraying a history of hardships but also a glimmer of hope.',
    choices=[
        'Offer your hand to help the last human and suggest forming a temporary alliance.',
        'Decide to leave the last human to fend for herself, opting to escape on your own.',
        'Propose a plan to together clear the area of zombines, seeking out supplies first.',
        'Ask the last human about her experiences and what she knows about the zombines.'
    ]
)

In [2]:
import asyncio
from pydantic import field_validator, ValidationInfo


class FinalStoryChoice(BaseModel):
    choice_description: str
    choice_consequences: str
    choices: list["FinalStoryChoice"]


class FinalStory(BaseModel):
    story_setting: str
    plot_summary: str
    choices: list[FinalStoryChoice]


class RewrittenChoice(BaseModel):
    choice_description: str
    choice_consequences: str
    choices: list[str]

    @field_validator("choices")
    def validate_choices(cls, v, info: ValidationInfo):
        original_choice = info.context["choice"]
        if original_choice in v:
            raise ValueError(
                "Choices must be unique - do not repeat the original choice"
            )
        return v


async def rewrite_choice(
    client: instructor.AsyncInstructor,
    choice: str,
    story: GeneratedStory,
    prev_choices: list[str],
    max_depth: int = 3,
) -> FinalStoryChoice:
    task = (
        "This is a terminal choice. End the story and rewrite the choice setting and consequence to reflect that. The story should be wrapped up here."
        if len(prev_choices) == max_depth - 1
        else "Continue the story and suggest 2-4 potential choices if applicable"
    )
    rewritten_choice = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": """
            You're an expert story writer. You're given the following choice from a story.

            Choice: {{ choice }}

            Rules
            - Make sure that the choice is consistent with the overall plot of the story.
            - {{task}}
            - If the story is ending, make sure that the story consequence reflects a proper wrap up to the story.
            - choice_description should be in the form of an action (Eg. "Move forward and do X"), be around 1-2 sentences and not mention the main character. It should only describe an action.

            - Choice descriptions should just be 1-2 sentences and reflect a specific user action that the user can take.
            - New Potential choices should be unique and distinct. They should move the story forward in a meaningful way and be distinct from the provided choice above since they are choices to be made after the user has made the choice above.
            - Feel free to terminate the story at this specific choice if it makes sense to do so. Just return an empty list for choices. 
            
            Story Description: 
            - Setting : {{ story.setting }}
            - Plot Summary : {{ story.plot_summary }}

            Prior Choices Made:
            {% for prev_choice in previous_choices %}
            {{ loop.index }}. {{ prev_choice['choice_consequences'] }} : {{ prev_choice['choice_description'] }}
            {% endfor %}
            """,
            }
        ],
        context={
            "choice": choice,
            "story": story,
            "previous_choices": prev_choices,
            "task": task,
        },
        response_model=RewrittenChoice,
    )

    if len(prev_choices) == max_depth - 1:
        return FinalStoryChoice(
            choice_description=rewritten_choice.choice_description,
            choice_consequences=rewritten_choice.choice_consequences,
            choices=[],
        )

    coros = [
        rewrite_choice(
            client,
            potential_choice,
            story,
            prev_choices
            + [
                {
                    "choice_description": rewritten_choice.choice_description,
                    "choice_consequences": rewritten_choice.choice_consequences,
                }
            ],
            max_depth,
        )
        for potential_choice in rewritten_choice.choices
    ]

    rewritten_choices = await asyncio.gather(*coros)

    return FinalStoryChoice(
        choice_description=rewritten_choice.choice_description,
        choice_consequences=rewritten_choice.choice_consequences,
        choices=rewritten_choices,
    )

In [14]:
async def rewrite_story(
    client: instructor.AsyncInstructor, story: GeneratedStory, max_depth: int = 3
):
    coros = [
        rewrite_choice(client, choice, story, [], max_depth) for choice in story.choices
    ]
    rewritten_choices = await asyncio.gather(*coros)
    return FinalStory(
        story_setting=story.setting,
        plot_summary=story.plot_summary,
        choices=rewritten_choices,
    )


rewritten_story = await rewrite_story(client, story, 2)

print(rewritten_story)

/var/folders/zs/vkrt6zx12rb6jfqz6r37fwhm0000gn/T/ipykernel_1309/1752687078.py:10: RuntimeWarning: coroutine 'rewrite_story' was never awaited
  rewritten_story = await rewrite_story(client,story,2)


FinalStory(
    story_setting='The world is cloaked in a thick veil of mist and decay, with ruined skyscrapers standing like 
skeletal remains against an overcast sky. Streets are choked with overgrown weeds and debris; the only sound is the
distant groaning of zombines echoing through the empty city. Faded billboards and rusted cars add to the eerie 
atmosphere, while the air is heavy with the scent of mold and rot. Despite the desolation, nature has begun to 
reclaim its territory, with vines creeping up the sides of buildings and trees sprouting through cracks in the 
pavement. In this forsaken landscape, you stumble upon a flicker of movement in the shadows. A small, flickering 
light catches your attention amid the chaos. As you approach, you realize it’s a makeshift shelter surrounded by a 
perimeter of scavenged materials, guarded against the constant threat of zombines.',
    plot_summary='As you navigate through the overgrown wreckage of the city, you hear a soft humming coming from a
nearby shelter, adorned with bits of reflective metal and scattered tools. Intrigued, you cautiously approach, your
heart pounding. Upon entering, you discover an older man, the last human alive, meticulously working on a 
contraption designed to fend off the zombines. His face, lined with age and wisdom, lights up when he sees you, 
hope sparking in his eyes. He tells you stories of survival, how he outsmarted the zombines and managed to live in 
solitude, but with his supplies dwindling, he’s starting to lose hope. As he speaks, a loud crash outside 
interrupts the moment, the sound of approaching zombines rattling the makeshift walls. You realize that your 
encounter with this last human could change the fate of survival for both of you.',
    choices=[
        FinalStoryChoice(
            choice_description='Offer to help him fortify the shelter against the zombines.',
            choice_consequences='Together, you and the old man gather materials and construct a sturdy barricade 
around the shelter. As night falls, the zombines approach but find their efforts thwarted by your defenses, giving 
you both a chance to survive another day.',
            choices=[
                FinalStoryChoice(
                    choice_description='Decide to search the nearby city for additional supplies to reinforce your 
defenses against the zombines.',
                    choice_consequences='In your search, you scavenge through the crumbling buildings and find 
essential supplies—a stash of old weapons, food rations, and tools that could greatly enhance your defenses. Upon 
returning, you and the old man fortify the shelter further, creating a secured haven amidst the chaos. As night 
falls, the zombines come, but with your newly acquired resources, you stand strong together. The two of you have 
formed an unbreakable bond of survival, facing the darkness with determination and hope as the world around you 
decays into oblivion.',
                    choices=[]
                ),
                FinalStoryChoice(
                    choice_description='Devise a clever distraction to draw the zombines away from the shelter, 
allowing you and the old man to escape unnoticed.',
                    choice_consequences='Your plan succeeds as you set off a chain of noises in the distance, 
successfully luring the zombines away. You and the old man clutch each other in suspense, then quietly slip out of 
the shelter, making your way through the foggy streets. As the mist envelops you, you reflect on the bond formed in
this bleak world and the hope of finding other survivors. Together, you embark on a new journey, ready to face 
whatever lies ahead, embracing the uncertainties of survival in a world reclaimed by nature and the undead.',
                    choices=[]
                ),
                FinalStoryChoice(
                    choice_description='Share stories of your past to strengthen the bond between you and the old 
man.',
              

In [16]:
from common.models import Story, StoryNode, JobStatus


Story = Story(
    title="Zombie Apocalypse",
    description=rewritten_story.story_setting,
    user_id="123",
    status=JobStatus.COMPLETED,
)


In [19]:
def count_nodes(nodes: list[FinalStoryChoice]):
    return len(nodes) + sum(count_nodes(node.choices) for node in nodes)

count_nodes(story.choices)

75

In [11]:
from pathlib import Path
import json

with open("./story.json") as f:
    rewritten_story = FinalStory(**json.load(f))

In [23]:
from typing import Optional
from uuid import UUID, uuid4
from common.models import StoryNode

def flatten_and_format_nodes(
    story_id,
    user_id,
    nodes: list[FinalStoryChoice],
    parent_id: Optional[UUID],
    acc: list[StoryNode],
) -> list[StoryNode]:
    for node in nodes:
        node_id = uuid4()
        new_node = StoryNode(
            id=node_id,
            story_id=story_id,
            parent_node_id=parent_id,
            choice_text=node.choice_description,
            image_url="",
            setting=node.choice_description,
            user_id=user_id,
        )
        acc = acc + [new_node]
        if node.choices:
            acc = flatten_and_format_nodes(
                story_id, user_id, node.choices, node_id, acc
            )
    return acc

nodes = flatten_and_format_nodes(
    "123",
    "123",
    rewritten_story.choices,
    None,
    [],
)



75

In [6]:
from pathlib import Path
import json

with open("./story.json") as f:
    story = FinalStory(**json.load(f))

FinalStory(story_setting="We're finally moving into the new place! The sun shines brightly in the clear blue sky, reflecting off the polished surfaces of the newly constructed home. The scent of fresh paint and cut grass pervades the air, creating a sense of anticipation. As we step through the front door, we're greeted by the echo of our footsteps on the hardwood floors and the smell of new beginnings. The living room is spacious, filled with sunlight that dances through the large windows. The walls are painted a soft cream, providing a warm, welcoming backdrop for the colorful memories we're about to create. Outside, a lush green yard stretches out, bordered by blooming flowers and an oak tree that promises to be a place for picnics and laughter. This is a blank canvas waiting for us to paint our lives upon it.", plot_summary="Today is the big day, the culmination of our long journey to find a perfect home. As we unload boxes and organize our belongings, excitement fills the air. Eve

In [7]:
def count_nodes(nodes: list[FinalStoryChoice]):
    return len(nodes) + sum(count_nodes(node.choices) for node in nodes)


print(count_nodes(story.choices))


75